In [1]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tqdm.notebook import tqdm
from time import time
import dataset
import model
import torch
import torch.utils.data as data

In [2]:
img_dir = '/opt/ml/input/data/train/images'
mean, std = (0.5, 0.5, 0.5), (0.2, 0.2, 0.2)
transform = dataset.get_transforms(mean=mean, std=std)

dataset = dataset.MaskBaseDataset(
    img_dir=img_dir,
)

# train dataset과 validation dataset을 8:2 비율로 나눕니다.
n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
train_dataset, val_dataset = data.random_split(dataset, [n_train, n_val])
train_dataset.dataset.set_transform(transform['train'])
val_dataset.dataset.set_transform(transform['val'])


In [3]:
train_loader = data.DataLoader(
    train_dataset,
    batch_size=128,
    num_workers=1,
    shuffle=True
)

val_loader = data.DataLoader(
    val_dataset,
    batch_size=128,
    num_workers=1,
    shuffle=False
)

In [4]:
images, labels = next(iter(train_loader))
print(f'images shape: {images.shape}')
print(f'labels shape: {labels.shape}')

images shape: torch.Size([128, 3, 512, 384])
labels shape: torch.Size([128])


In [5]:
imgnet_resnet18 = model.get_imagenet_resnet18()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
imgnet_resnet18.to(device)

print(f"{device} is using!")

LEARNING_RATE = 0.001
NUM_EPOCH = 10

loss_fn = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(imgnet_resnet18.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train" : train_loader,
    "test" : val_loader
}

cuda:0 is using!


In [6]:
best_test_accuracy = 0.
best_test_loss = 9999.
for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      imgnet_resnet18.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      imgnet_resnet18.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = imgnet_resnet18(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")


현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.444, 평균 Accuracy : 0.854



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.503, 평균 Accuracy : 0.819



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.169, 평균 Accuracy : 0.943



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.396, 평균 Accuracy : 0.871



현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.077, 평균 Accuracy : 0.975


...


현재 epoch-7의 test-데이터 셋에서 평균 Loss : 0.129, 평균 Accuracy : 0.961



현재 epoch-8의 train-데이터 셋에서 평균 Loss : 0.015, 평균 Accuracy : 0.995



현재 epoch-8의 test-데이터 셋에서 평균 Loss : 0.149, 평균 Accuracy : 0.952



현재 epoch-9의 train-데이터 셋에서 평균 Loss : 0.016, 평균 Accuracy : 0.995



현재 epoch-9의 test-데이터 셋에서 평균 Loss : 0.116, 평균 Accuracy : 0.963
학습 종료!
최고 accuracy : 0.9631271958351135, 최고 낮은 loss : 0.1158148600870399


In [7]:
class TestDataset(data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [8]:
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

test_dir = '/opt/ml/input/data/eval'

submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
test_image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
test_image_paths = [os.path.join(test_image_dir, img_id) for img_id in submission.ImageID]
test_transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

test_dataset = TestDataset(test_image_paths, test_transform)

test_loader = data.DataLoader(
    test_dataset,
    shuffle=False
)

imgnet_resnet18.eval()

all_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        pred = imgnet_resnet18(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
